<a href="https://colab.research.google.com/github/shubhomb/greenstand_data_analysis/blob/master/imnet/bounding_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [ ]:

# how to access GDrive https://colab.research.google.com/notebooks/io.ipynb#scrollTo=RWSJpsyKqHjH
from google.colab import files, drive
import os
drive.mount('/content/drive')
gdir = os.path.join(os.getcwd(), "drive", "My Drive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AFEkI9BC8sfUcEpbVJJOtOrZXP0rO_fl3LYAMbs45z0Ya5cT77ljnQ
Mounted at /content/drive


## Dependencies

In [ ]:
# Torch Dataset and IMNet Loading
import torch
from xml.etree import ElementTree
from torch.utils import data
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from PIL import Image, ImageDraw
from  collections import OrderedDict
import numpy as np

# Model development and training
import torchvision.models as models
import torch.nn as nn

# Filesystem and parallelization
import os
import multiprocessing

# Utility 
import time
import datetime

# Constants for parallelization
_DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# from ImageNet
synsets = {
    "judas": "n12513613",
    "palm": "n12582231",
    "pine": "n11608250",
    "china tree": "n12741792",
    "fig": "n12401684",
    "cabbage": "n12478768",
    "cacao": "n12201580",
    "kapok": "n12190410",
    "iron": "n12317296",
    "linden": "n12202936",
    "pepper": "n12765115",
    "rain": "n11759853",
    "dita": "n11770256",
    "alder": "n12284262",
    "silk": "n11759404",
    "coral": "n12527738",
    "huisache": "n11757851",
    "fringe": "n12302071",
    "dogwood": "n12946849",
    "cork": "n12713866",
    "ginkgo": "n11664418",
    "golden shower": "n12492106",
    "balata": "n12774299",
    "baobab": "n12189987",
    "sorrel": "n12242409",
    "Japanese pagoda": "n12570394",
    "Kentucky coffee": "n12496427",
    "Logwood": "n12496949"
}


print ("Device:" , _DEVICE)

Device: cuda:0



## Dataset Creation
Define datasets for ImageNet and Greenstand sources. Greenstand species classes are yet unlabeled.

In [ ]:
class ImnetDataset(data.Dataset):
    
    # initialise function of class
    def __init__(self, dir, synsets, transforms=None, device=None, one_hot=False):
        # the data directories
        self.img_dir = os.path.join(dir, "original_images")
        self.bb_dir = os.path.join(dir, "bounding_boxes")

        #synsets library to get the associated class
        self.synsets = synsets
        self.rev_synsets = {y:x for x,y in zip(synsets.keys(), synsets.values())}
        self.classes = list(self.synsets.keys())
        # self.target = target

        self.one_hot = one_hot
        self.imgs = []

        for i in self.classes:
          temp_imgs = list(sorted(os.listdir(os.path.join(self.img_dir, i))))
          for img_path in temp_imgs:
            #in every directory the "tar" file is still present
            if not "tar" in img_path:
              name = img_path.split('.')[0]
              self.imgs.append(name)

        self.bb_dict = {}
        for f, _, d in os.walk(self.bb_dir):
          for file in d:
            if os.path.splitext(file)[1] == ".xml":
              tree = ElementTree.parse(os.path.join(f, file))
              root = tree.getroot()
              obj = root.find("object")
              b = obj.find("bndbox")
              xmin = int(b.find("xmin").text)
              ymin = int(b.find("ymin").text)
              xmax = int(b.find("xmax").text)
              ymax = int(b.find("ymax").text)
              self.bb_dict[os.path.join(f, file)] =  (xmin, ymin, xmax, ymax)

        self.transforms = transforms
        self.device = device
        

    def __getitem__(self, idx):
        name = self.imgs[idx]
        label = self.rev_synsets[name.split("_")[0]]

        img_path = os.path.join(self.img_dir, label, f"{name}.JPEG")
        bb_path = os.path.join(self.bb_dir, label, "Annotation", name.split("_")[0], f"{name}.xml")
        img = Image.open(img_path).convert("RGB")


        if bb_path in self.bb_dict.keys():
          xmin, ymin, xmax, ymax = self.bb_dict[bb_path]
        else:
          # the whole image is the bounding box label, as NoneType was causing collating issue. 
          xmin = 0
          ymin = 0
          xmax = img.size[0]
          ymax = img.size[1]
        boxes = torch.as_tensor([xmin, ymin, xmax, ymax], dtype=torch.float32)

        
        # img, boxes = self.expand(img, [xmin, ymin, xmax, ymax], self.target, (0, 0, 0))
        if self.transforms is not None:
          img = self.transforms(img)

        if self.one_hot: 
          image_id = torch.zeros(len(self.classes), dtype=torch.float32)
          image_id [self.classes.index(label)] = 1.0
        else:
          image_id = torch.tensor([self.classes.index(label)])

        targets = {}
        targets["boxes"] = boxes
        targets["image_class"] = image_id

    
        return img, targets

    def __len__(self):
        return len(self.imgs)


In [ ]:
class GreenstandDataset(data.Dataset):
  # We don't have labels for this yet...
  def __init__(self, dir, device, transforms, bb_dir=None, one_hot=False):
    self.img_dir = dir
    self.imgs = []
    self.bb_dir = bb_dir
    self.transforms = transforms
    self.classes = None # Change this when we define Greenstand class labels
    self.one_hot = one_hot
    self.device = device

    for f, _, d in os.walk(test_path):
      for fil in d:
        if "jpg" in fil:
          self.imgs.append(os.path.join(f, fil))

  def __getitem__(self, idx):
    img = Image.open(self.img_dir[idx])
    if self.transforms is not None:
      img = self.transforms (img)


  def __len__(self):
    return len(self.imgs)

## MobileNet-v2
See [the original paper](https://arxiv.org/pdf/1704.04861.pdf) for details. This was chosen first because Torchvision has pretrained weights and the net is quite low-latency, which may be useful for user-interface image selection. First go is to simply change the output layer to predict 4 coordinates for the bounding box.

In [ ]:
mobilenet = models.mobilenet_v2(pretrained=True)
# Preprocessing required for MobileNet v2

    
def expand(self, pil_img, boxes, target, background_color):
    size = {}
    size["width"], size["height"] = pil_img.size
    if max(size) == "width":
        size["new_height"] = int(target/size["width"] * size["height"])
        size["new_width"] = int(target)
    else:
        size["new_width"] = int(target/size["height"] * size["width"])
        size["new_height"] = int(target)


    x_scale = size["new_width"]/size["width"]
    y_scale = size["new_height"]/size["height"]

    pil_img = pil_img.resize((size["new_width"], size["new_height"]))

    xmin = boxes[0]
    ymin = boxes[1]
    xmax = boxes[2]
    ymax = boxes[3]

    xmin = int(np.round(xmin*x_scale)) + (target - size["new_width"])/2
    ymin = int(np.round(ymin*y_scale)) + (target - size["new_height"])/2
    xmax= int(np.round(xmax*(x_scale))) + (target - size["new_width"])/2
    ymax= int(np.round(ymax*y_scale)) + (target - size["new_height"])/2

    result = Image.new(pil_img.mode, (target, target), background_color)
    result.paste(pil_img, (0, (size["new_width"] - size["new_height"]) // 2))

    return result, torch.as_tensor([xmin, ymin, xmax, ymax], dtype=torch.float32)

mobilenet_preprocessing = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])




Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


### Instantiate datasets, define loader processes

In [ ]:
path = "/content/drive/My Drive/data/imnet"
test_path = "/content/drive/My Drive/data/test_greenstand_samples"
model_path = "/content/drive/My Drive/models/ImageNet/mobilenet/%s"%datetime.datetime.today().date()
if not os.path.exists(model_path):
  os.makedirs(model_path)



In [ ]:
start = time.time()
data_set = ImnetDataset(path, synsets, transforms=mobilenet_preprocessing, one_hot=False, device=_DEVICE)
greenstand_test = GreenstandDataset(test_path, transforms=mobilenet_preprocessing, one_hot=False, device=_DEVICE)

print ("Finished creating datasets in ", time.time() - start, " seconds ") # this can take ~30 minutes

In [ ]:
# Helper functions 
def rmse(x, y):
  '''
  Root-mean squared error of two vectors of the same batch 
  '''
  return torch.sqrt( (1/x.size()[0]) * torch.sum((x-y) **2))

def iou(box_a, box_b):
  # order is xmin, ymin, xmax, ymax 
  intersect_xmin = max(box_a[0], box_b[0])
  intersect_ymin = max(box_a[1], box_b[1])
  intersect_xmax = min(box_a[2], box_b[2])
  intersect_ymax = min(box_a[3], box_b[3])
  area_intersect = max(0, intersect_xmax - intersect_xmin) * max(0, intersect_ymax - intersect_ymin)

  area_a = (box_a[3] - box_a[1]) * (box_a[2] - box_a[0])
  area_b = (box_b[3] - box_b[1]) * (box_b[2] - box_b[0])
  union = area_a + area_b - area_intersect
  return area_intersect / union

class Customized_MobileNet(nn.Module):
  def __init__(self, pretrained_model, num_classes=10):
    super().__init__()
    self.pretrained = pretrained_model
    for param in self.pretrained.parameters():
      param.requires_grad = False
    self.pretrained.classifier = nn.Identity()
    self._classifier_layer(num_classes)
    self._regressor_layer()

    
  def forward(self, x):
    """
    The model is performing a regression on bounding boxes and a classifier 
    """
    return (self.classifier(self.pretrained(x)), self.regressor(self.pretrained(x)))

  def _classifier_layer(self, num_classes):
    """
    Initializes final classification layer for labeling genus, species, etc.
    """
    self.classifier = nn.Sequential(
                        nn.Dropout(0.2),
                        nn.Linear(1280, num_classes) # 1280 is num_outputs of the last feature layer
                      ) 
    for param in self.classifier.parameters():
      param.requires_grad=True

  def _regressor_layer(self):
    """
    A bounding box output layer for predicting object location
    This is currently designed to output exactly one bounding box
    """
    self.regressor =  nn.Sequential(
                        nn.Dropout(0.2), 
                        nn.Linear(1280, 4) # 1280 is num_outputs of the last feature layer
                      )
    for param in self.regressor.parameters():
      param.requires_grad=True

class ModelTrainer():
  '''
  An abstraction to help keep track of model parameters and run training. 
  '''
  def __init__ (self, model, dataset, learning_rate, alpha, beta, device, batch_size, model_savepath,
                gamma=1e-4, train_split=0.8, pin_memory=False, n_workers=0):
    
    self.model = model # like Customized_MobileNet
    self.model_savepath = os.path.join (model_savepath, "checkpoint.pth.tar")
    # Initialize device
    self.device = device
    if self.device == torch.device("cuda:0"):
      self.model.cuda()

    # Make validation split
    self.trainsize = int(train_split * len(dataset))
    self.valsize = len(dataset) - self.trainsize
    train_dataset, valid_dataset = torch.utils.data.dataset.random_split(dataset, [self.trainsize, self.valsize])

    # Define data loader for training and validation
    self.batch_size = batch_size
    self.data_loader  = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, sampler=None,
              batch_sampler=None, num_workers=n_workers, collate_fn=None,
              pin_memory=pin_memory, drop_last=False, timeout=0,
              worker_init_fn=None)

    self.val_data_loader = DataLoader(valid_dataset, batch_size=self.batch_size, shuffle=True, sampler=None,
              batch_sampler=None, num_workers=n_workers, collate_fn=None,
              pin_memory=pin_memory, drop_last=False, timeout=0,
              worker_init_fn=None)
    
    # Loss specifications and optimizer parameter setting
    # This is defined here so that the underlying model can be changed (i.e. hidden layers) 
    self.alpha, self.beta = alpha, beta # classification/regression loss tradeoff
    cps = [param for param in self.model.classifier.parameters()]
    rps = [param for param in self.model.regressor.parameters()]
    self.optimizer = torch.optim.Adam(params=cps+rps, lr=learning_rate, weight_decay=gamma)
    self.regression_criterion = nn.MSELoss()
    self.classification_criterion = nn.CrossEntropyLoss()

    if os.path.exists(self.model_savepath):
      print ("Found saved model at savepath %s" %(self.model_savepath))
      checkpoint = torch.load(self.model_savepath)
      self.model.load_state_dict(checkpoint['model_state_dict'])
      self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      self.start_epoch = checkpoint['epoch']
    else:
      self.start_epoch = 0


  def describe_training(self):
    print (self.trainsize, " training examples")
    print (self.valsize, " validation examples")

  def train(self, num_epochs, val_interval=1, batch_report=50, batch_lookback=10):
    '''
    Main function to train. 
    @param num_epochs(int): Number of epochs of training
    @param val_interval(int): Interval epochs between validation metric
    @param batch_report(int): Interval batches between training reports
    @param batch_lookback(int): Number of batches to use for averaging metrics in printing
    '''
    num_tr_batches = np.ceil(self.trainsize/self.data_loader.batch_size)
    num_val_batches = np.ceil(self.valsize/self.valsize)
    epoch_loss = []
    epoch_acc = []
    epoch_rmse = []
    epoch_iou = []
    val_epoch_loss = []
    val_epoch_acc = []
    val_epoch_rmse = []
    val_epoch_iou = []
    print ("Starting at epoch %d"%self.start_epoch)
    for epoch in range(self.start_epoch, num_epochs):
      epoch_start = time.time()
      print ("=" * 50)
      print ("EPOCH ", epoch)
      batch_count = 0
      batch_loss = []
      batch_acc = []
      batch_rmse = []
      batch_iou = []
      for batchx, batchy in self.data_loader:
          batch_count += 1
          # Device designation
          if self.device == torch.device("cuda:0"):
            batchx = batchx.cuda(non_blocking=True)
            batchy["boxes"] = batchy["boxes"].cuda(non_blocking=True)
            batchy["image_class"] = batchy["image_class"].cuda(non_blocking=True)
          
          class_labels = batchy["image_class"]
          box_labels = batchy["boxes"]

          # Forward pass
          class_preds, box_preds = self.model.forward(batchx)
          loss = self._loss_specification(class_preds, class_labels, box_preds, box_labels)
          loss.backward()
          self.optimizer.step()

          # Metrics 
          box_rmse = rmse(box_preds, box_labels)
          avg_box_iou = torch.mean(torch.as_tensor([iou(box_labels[i, :], box_preds[i, :]) for i in range(box_labels.size()[0])], dtype=torch.float32))
          class_correct = (torch.max(class_preds, 1)[1] == class_labels.squeeze()).sum()
          acc = class_correct/float(batchx.shape[0])
          batch_iou.append(avg_box_iou)
          batch_rmse.append(box_rmse)
          batch_acc.append(acc)
          batch_loss.append(loss.data)

          if batch_count % batch_report == 0 or batch_count == num_tr_batches:
            print ("\nLast %d Batch Avg Metrics, Batch %d/%d" %(batch_lookback, batch_count, num_tr_batches))
            print ("Total Loss: {:.3f}".format(torch.mean(torch.as_tensor(batch_loss[-batch_lookback:], dtype=torch.float32))))
            print ("Classification Acc: {:.3f}".format(torch.mean(torch.as_tensor(batch_acc[-batch_lookback:], dtype=torch.float32))))
            print ("BBox RMSE: {:.3f}".format(torch.mean(torch.as_tensor(batch_rmse[-batch_lookback:], dtype=torch.float32))))
            print ("Avg Bbox IoU: {:.3f} \n".format(torch.mean(torch.as_tensor(batch_iou[-batch_lookback:], dtype=torch.float32))))
            torch.save({
                          'epoch': epoch + 1,
                          'model_state_dict': self.model.state_dict(),
                          'optimizer_state_dict': self.optimizer.state_dict(),
                          }, 
                       self.model_savepath)
            print ("Checkpoint created")
      
      if epoch % val_interval == 0:
          print ("VALIDATION EPOCH ", epoch)
          batch_count = 0
          
          self.model.eval()
          with torch.no_grad():
            rmses = []
            ious = []
            losses = []
            class_accs = []

            for batchx, batchy in self.val_data_loader:
                batch_count += 1
                 # Device designation
                if self.device == torch.device("cuda:0"):
                  batchx = batchx.cuda(non_blocking=True)
                  batchy["boxes"] = batchy["boxes"].cuda(non_blocking=True)
                  batchy["image_class"] = batchy["image_class"].cuda(non_blocking=True)
                
                class_labels = batchy["image_class"]
                box_labels = batchy["boxes"]
                class_preds, box_preds = self.model.forward(batchx)
                losses.append(self._loss_specification(class_preds, class_labels, box_preds, box_labels).data)
                class_accs.append(float((torch.max(class_preds, 1)[1] == class_labels.squeeze()).sum())/self.val_data_loader.batch_size)
                ious.append(torch.mean(torch.as_tensor([iou(box_labels[i, :], box_preds[i, :]) for i in range(box_labels.size()[0])], dtype=torch.float32)))
                rmses.append(rmse(box_preds, box_labels))

              
            losses = torch.mean(torch.as_tensor(losses, dtype=torch.float32))
            class_accs = torch.mean(torch.as_tensor(class_accs, dtype=torch.float32))
            box_rmse = torch.mean(torch.as_tensor(rmses, dtype=torch.float32))
            avg_box_iou = torch.mean(torch.as_tensor(ious, dtype=torch.float32))
            val_epoch_loss.append(losses)
            val_epoch_acc.append(class_accs)
            val_epoch_rmse.append(box_rmse)
            val_epoch_iou.append(avg_box_iou)

            # We can change this to be epoch wise or not averaged over all batches
            print ("Batch Average Val Loss: {:.3f}".format(losses))
            print ("Batch Avg Val Classification Acc: {:.3f}".format(class_accs))
            print ("Batch Avg Val BBox RMSE: {:.3f}".format(box_rmse))
            print ("Batch Avg Avg Bbox IoU: {:.3f} \n".format(avg_box_iou))
          self.model.train()
      epoch_loss.append(torch.mean(torch.as_tensor(batch_loss, dtype=torch.float32)))
      epoch_acc.append(torch.mean(torch.as_tensor(batch_acc, dtype=torch.float32)))
      epoch_iou.append(torch.mean(torch.as_tensor(batch_iou, dtype=torch.float32)))
      epoch_rmse.append(torch.mean(torch.as_tensor(batch_rmse, dtype=torch.float32)))
  
      print ("Epoch ", epoch + 1, " finished in ", time.time() - epoch_start)
    tr_metric_dict = {"Loss": epoch_loss, "Acc": epoch_acc, "IoU": epoch_iou, "RMSE": epoch_rmse}
    val_metric_dict = {"Loss": val_epoch_loss, "Acc": val_epoch_acc, "IoU": val_epoch_iou, "RMSE": val_epoch_rmse}
    torch.save({
              'epoch': epoch + 1,
              'model_state_dict': self.model.state_dict(),
              'optimizer_state_dict': self.optimizer.state_dict(),
              'tr_metric_dict': tr_metric_dict, 
              'val_metric_dict': val_metric_dict
              }, 
            self.model_savepath)
    print ("Final checkpoint created. Model dict and metrics saved. ")
    return self.model, tr_metric_dict, val_metric_dict


  def _loss_specification(self, class_preds, class_labels, box_preds, box_labels):
    bounding_box_error = self.regression_criterion(box_preds, box_labels) # bounding box regression error
    # preds_flat = torch.max(class_preds, dim=1)[1] # BSIZE x 1 (index)
    classification_loss = self.classification_criterion(class_preds, class_labels.view(-1)) # classification error
    return self.alpha * classification_loss + self.beta * bounding_box_error

In [ ]:
torch.multiprocessing.set_start_method("spawn", force=True)

_N_WORKERS = 0
_PIN_MEM = True

mob_model = Customized_MobileNet(pretrained_model=mobilenet, num_classes=28)
trainer = ModelTrainer(mob_model, 
                       data_set, 
                       train_split=0.8,
                       learning_rate=0.002,
                       batch_size = 32,
                       alpha=0.2, # classification loss weight 
                       beta=0.8, # regression loss weight
                       device=_DEVICE, 
                       pin_memory=_PIN_MEM,
                       n_workers=_N_WORKERS,
                       model_savepath=model_path)
print (trainer.describe_training())


23085  training examples
5772  validation examples
None


### Check model structure and parameters
Regressor and classifier (final layer) should require grad. Others should not. Optimizer should be set only to those regressor and classifier variables. 

In [ ]:
print ("Regressor params")
print ([p for p in mob_model.regressor.parameters()])

print ("Classifier params")
print ([p for p in mob_model.classifier.parameters()])

# Should output just two layers (4 variables total)
for param in mob_model.parameters():
  if param.requires_grad:
    print (param.shape)

Regressor params
[Parameter containing:
tensor([[ 1.6537e-02, -1.0734e-02, -1.0267e-02,  ...,  2.4273e-02,
         -1.0482e-02, -2.4949e-02],
        [-1.8666e-02,  4.3694e-03,  1.4824e-02,  ...,  4.8098e-03,
          8.4475e-03, -6.0068e-04],
        [ 2.6587e-03, -2.8985e-03,  7.2081e-03,  ...,  1.3932e-02,
          4.9859e-03, -1.8791e-02],
        [-2.5874e-02,  8.8736e-03,  1.4604e-02,  ...,  3.8725e-05,
         -5.7580e-03, -1.4739e-02]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0137, 0.0058, 0.0021, 0.0037], device='cuda:0', requires_grad=True)]
Classifier params
[Parameter containing:
tensor([[-0.0219,  0.0032, -0.0052,  ..., -0.0260, -0.0169, -0.0185],
        [-0.0229,  0.0061, -0.0253,  ..., -0.0146,  0.0052,  0.0035],
        [ 0.0228, -0.0158, -0.0019,  ..., -0.0021,  0.0033,  0.0258],
        ...,
        [-0.0093,  0.0067,  0.0028,  ..., -0.0248, -0.0062,  0.0035],
        [-0.0130,  0.0240,  0.0254,  ..., -0.0072, -0.0142,  0.0142],
    

In [ ]:
trainer.train(num_epochs=10,batch_report=50) 

Starting at epoch 0
EPOCH  0

Last 10 Batch Avg Metrics, Batch 50/722
Total Loss: 78944.727
Classification Acc: 0.100
BBox RMSE: 624.127
Avg Bbox IoU: 0.056 

Checkpoint created
